In [1]:
import requests
import urllib
import json
import os 
import pandas as pd
import numpy as np
import urllib.parse
from bs4 import BeautifulSoup

In [2]:
# 搜尋音樂
def get_song_list(keyword):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
    keyword = urllib.parse.quote(keyword)
    url = 'https://c.y.qq.com/soso/fcgi-bin/client_search_cp?aggr=1&cr=1&p=1&n=20&w=%s' % keyword
    response = requests.get(url, headers=headers).text.encode(
        'gbk', 'ignore').decode('gbk').split('callback')[-1].strip('()')
    response = json.loads(response)
    return response['data']['song']

In [3]:
def print_info(songs):
    for num, song in enumerate(songs):
        songname = song['songname']
        singer_length = len(song['singer'])
        singers = []
        for i in range(singer_length):
            singers.append(song['singer'][i]['name'])
        singers = ('/').join(singers)
        media_mid = song['media_mid']
        album_name = song['albumname']
        time = song['interval']
        m, s = divmod(time, 60)
        time = "%02d:%02d" % (m, s)
        print(num,'Song：',songname,'Artist：' ,singers, 'Album：',album_name, 'Duration：',time)

In [4]:
def get_mp3_url(songs,num):
    media_mid = songs['list'][num]['media_mid']
    url_1 = 'https://c.y.qq.com/base/fcgi-bin/fcg_music_express_mobile3.fcg?g_tk=5381&cid=205361747&songmid=%s&filename=C400%s.m4a&guid=6800588318' % (media_mid,media_mid)
    response = requests.get(url_1).json()
    vkey= response['data']['items'][0]['vkey']
    if vkey:
        url_2 = 'http://dl.stream.qqmusic.qq.com/C400%s.m4a?vkey=%s&guid=6800588318&uin=0&fromtag=66' % (media_mid,vkey)
        return url_2
    return None

In [11]:
def download_mp3(url,filename):
    abspath = os.path.abspath('.')  # 獲取絕對路徑
    os.chdir(abspath)
    response = requests.get(url).content
    path = os.path.join(abspath,filename)
    with open(filename + '.m4a', 'wb') as f:
        f.write(response)
        print('Download Finisg, save in %s' % path + '.m4a') 

In [14]:
def run():
    while  True:
        name = input('Input Song：')
        songs = get_song_list(name)
        if songs['totalnum'] == 0 : 
            print('No song found, please change keywords')
        else:
            print_info(songs['list'])
            num = input('Please enter the song you want to download:')
            url = get_mp3_url(songs,int(num))
            print(url)
            if not url :
                print('Song is no longer available. Download address not found. Download failed.')
            else:
                songname = songs['list'][int(num)]['songname']
                download_mp3(url, songname)

            flag = input('If you want to continue, you can press any key to search the song, otherwise press 0 to end the program')
            if flag == '0':
                break

In [ ]:
run()